In [1]:
from collections import Counter
import numpy as np
from io import BytesIO
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import cv2
import requests
import json
from unicodedata import normalize
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,ConfusionMatrixDisplay
from tensorflow.keras.layers import Input, Dense,Concatenate, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D,Dropout, Add,Embedding
from keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


Num GPUs Available:  1


In [2]:
MAX_URL_LEN_CHAR = 100
MAX_URL_LEN_WORD = 100
MAX_WORD_LEN = 20
int_to_class = {0: 'Adult', 1: 'Benign', 2: 'Malicious'}
img_int_to_class = {0:'nude', 1:'safe', 2:'unsafe'}
decode = lambda int_: int_to_class[int_]

decode_img_class = lambda img_class_int: img_int_to_class[img_class_int]
def text_to_word_sequence(
    input_text,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=" ",
):
    
    if lower:
        input_text = input_text.lower()

    translate_dict = {c: split for c in filters}
    translate_map = str.maketrans(translate_dict)
    input_text = input_text.translate(translate_map)

    seq = input_text.split(split)
    return [i for i in seq if i]

In [3]:
with open('./tokenizers/char_tokenizer.json') as f:
    data = json.load(f)
    char_tokenizer = tokenizer_from_json(data)

with open('./tokenizers/word_tokenizer.json') as f:
    data = json.load(f)
    word_tokenizer = tokenizer_from_json(data)

In [4]:
class Url_preprocessor():
    def __init__(self, char_tokenizer, word_tokenizer):
        self.char_tokenizer = char_tokenizer
        self.word_tokenizer = word_tokenizer
        
    def _preprocess_char_input(self, url):
        char_sequences = self.char_tokenizer.texts_to_sequences(url)
        return pad_sequences(char_sequences, maxlen=MAX_URL_LEN_CHAR, padding='post',truncating='post')
    
    def _preprocess_word_input(self, url):
        word_sequences = self.word_tokenizer.texts_to_sequences(url)
        return pad_sequences(word_sequences, maxlen=MAX_URL_LEN_WORD, padding='post',truncating='post')
    
    def _preprocess_word_char_input(self, url):
        sentences_splitted = [text_to_word_sequence(sentence) for sentence in url]
    
        for i in range(len(sentences_splitted)):
            if len(sentences_splitted[i])>MAX_URL_LEN_CHAR:
                sentences_splitted[i] = sentences_splitted[i][:MAX_URL_LEN_CHAR]
            else:
                while len(sentences_splitted[i])<MAX_URL_LEN_CHAR:
                    sentences_splitted[i].append('<OOV>')
            
        words_splitted = [[word.split() for word in sentence] for sentence in sentences_splitted]
    
        word_char_sequences = []
        
        for sentence in words_splitted:
            sentence_tokenized = []
            for i in range(len(sentence)):
                word = self.char_tokenizer.texts_to_sequences(sentence[i])
                word_char_padded = pad_sequences(word,maxlen=MAX_WORD_LEN,padding='post',truncating='post')
                sentence_tokenized.append(word_char_padded)
            word_char_sequences.append(sentence_tokenized)
        try:
            word_char_sequences = np.array(word_char_sequences)
        except:
            print(url)
        try:
            word_char_sequences = word_char_sequences.reshape(word_char_sequences.shape[0],word_char_sequences.shape[1],word_char_sequences.shape[-1])
        except:
            print(url)
        return word_char_sequences

    def preprocess(self,url):
        url = [url]
        char_input_data = self._preprocess_char_input(url)
        word_input_data = self._preprocess_word_input(url)
        word_char_input_data = self._preprocess_word_char_input(url)
        
        return [char_input_data, word_input_data, word_char_input_data]
    

In [3]:
def predict_url(url):
    url_preprocessor = Url_preprocessor(char_tokenizer,word_tokenizer)
    url_to_predict = url_preprocessor.preprocess(url)
    prediction = decode(model.predict(url_to_predict).argmax())
    print(prediction)
    return prediction

In [4]:
def load_model(model_name):
    model = tf.keras.models.load_model(model_name)
    return model

In [7]:
model = load_model('best_model.h5')

In [17]:
predict_url('http://architecture.com/')

Malicious


'Malicious'

# MobileNet Predictions

In [19]:
import requests
from io import BytesIO
import matplotlib.image as mpimg
mobilenet = load_model('mobilenet.h5')

def read_img_src(url):
    response = requests.get(url)
    img = mpimg.imread(BytesIO(response.content), format='JPG')
    # plt.imshow(img)
    # plt.axis('off')  # Turn off axis
    # plt.show()
    return img

def predict_img(url_src,model):
    img = read_img_src(url_src)
    resize = tf.image.resize(img, (128,128))
    yhat = model.predict(np.expand_dims(resize/255, 0))
    yhat = yhat.argmax()
    print(f'predicted class is: {decode_img_class(yhat)}')
    return decode_img_class(yhat)

In [14]:
img = read_img_src("https://scontent.xx.fbcdn.net/v/t1.15752-9/429102256_944359293332681_5837751611829704724_n.jpg?stp=dst-jpg_s851x315&_nc_cat=106&ccb=1-7&_nc_sid=5f2048&_nc_eui2=AeEUmA_6VukR6C4KfVVwkzNyvQWo3sseoky9Bajeyx6iTBsjBz-lbsriaLhMGT5HwunEAiqkeQ-CUYmu0ElPYxb-&_nc_ohc=aAvWhhMUzcMQ7kNvgGubamF&_nc_ad=z-m&_nc_cid=0&_nc_ht=scontent.xx&oh=03_Q7cD1QFbun4C4fQnvW7HP2RGPeTvN7OPlpFBmYN5Pt_WeX4pbQ&oe=669F28F4")

In [20]:
img = "https://scontent.xx.fbcdn.net/v/t1.15752-9/429102256_944359293332681_5837751611829704724_n.jpg?stp=dst-jpg_s851x315&_nc_cat=106&ccb=1-7&_nc_sid=5f2048&_nc_eui2=AeEUmA_6VukR6C4KfVVwkzNyvQWo3sseoky9Bajeyx6iTBsjBz-lbsriaLhMGT5HwunEAiqkeQ-CUYmu0ElPYxb-&_nc_ohc=aAvWhhMUzcMQ7kNvgGubamF&_nc_ad=z-m&_nc_cid=0&_nc_ht=scontent.xx&oh=03_Q7cD1QFbun4C4fQnvW7HP2RGPeTvN7OPlpFBmYN5Pt_WeX4pbQ&oe=669F28F4"

In [21]:
predict_img(img,mobilenet)

predicted class is: safe


'safe'